# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from scipy import stats as st
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.dummy import DummyClassifier
from sklearn.utils import shuffle

In [ ]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
df_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


## Обучение и проверка модели

Так как регионы никак не названы я дам им кодовые имена для того чтобы было легче структурировать проект и ориентироваться.
<br> <b>Центр 
<br> Юг
<br> Север

In [ ]:
def LinearRegression_func(features_train,target_train,features_valid,target_valid):
    model = LinearRegression()
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    df = pd.DataFrame(features_valid)
    df['model.predict'] = predictions_valid
    df = pd.merge(target_valid,df[['model.predict']],how = 'left',left_index = True, right_index = True)
    return [predictions_valid,model,df]

### Центр

In [ ]:
df_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
features = df_0.drop(['product','id'], axis=1)
target = df_0['product']

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(features,target, test_size=0.25, random_state=12345)

In [ ]:
model_center = LinearRegression_func(features_train,target_train,features_valid,target_valid)
mse = mean_squared_error(target_valid, model_center[0])
mean_center = model_center[0].mean()
print("RMSE =", mse ** 0.5)
print('Cредний запас предсказанного сырья ',mean_center)

RMSE = 37.5794217150813
Cредний запас предсказанного сырья  92.59256778438035


In [ ]:
model_center[2]

,product,model.predict
71751,10.038645,95.894952
80493,114.551489,77.572583
2655,132.603635,77.892640
53233,169.072125,90.175134
91141,122.325180,70.510088
...,...,...
12581,170.116726,103.037104
18456,93.632175,85.403255
73035,127.352259,61.509833
63834,99.782700,118.180397


### Юг

In [ ]:
features = df_1.drop(['product','id'], axis=1)
target = df_1['product']

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(features,target, test_size=0.25, random_state=12345)

In [ ]:
model_south = LinearRegression_func(features_train,target_train,features_valid,target_valid)
mse = mean_squared_error(target_valid, model_south[0])
mean_south = model_south[0].mean()
print("RMSE =", mse ** 0.5)
print('Cредний запас предсказанного сырья',mean_south)

RMSE = 0.893099286775617
Cредний запас предсказанного сырья 68.728546895446


### Север

In [ ]:
features = df_2.drop(['product','id'], axis=1)
target = df_2['product']

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(features,target, test_size=0.25, random_state=12345)

In [ ]:
model_nort = LinearRegression_func(features_train,target_train,features_valid,target_valid)
mse = mean_squared_error(target_valid, model_nort[0])
mean_nort = model_nort[0].mean()
print("RMSE =", mse ** 0.5)
print('Cредний запас предсказанного сырья',mean_nort)

RMSE = 40.02970873393434
Cредний запас предсказанного сырья 94.96504596800489


In [ ]:
model_nort[2]

,product,model.predict
71751,61.212375,93.599633
80493,41.850118,75.105159
2655,57.776581,90.066809
53233,100.053761,105.162375
91141,109.897122,115.303310
...,...,...
12581,28.492402,78.765887
18456,21.431303,95.603394
73035,125.487229,99.407281
63834,99.422903,77.779912


## Вывод

Наибольшие запасы нефти находятся в северном регионе. Самая лучшая модель получилась на южном регионе.

## Подготовка к расчёту прибыли

<br> Бюджет на разработку скважин в регионе — 10 млрд рублей.
3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.

Подготовьтесь к расчёту прибыли:
3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.
3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
3.3. Напишите выводы по этапу подготовки расчёта прибыли.

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

In [ ]:
budget = 10000000000
barrel_price = 450000
research_points = 500
count_of_wells = 200
price_one_well = budget / count_of_wells
break_even_barrel = price_one_well / barrel_price

In [ ]:
print('Стоимость разработки одной скважины:',price_one_well)
print('Достаточный объём сырья для безубыточной разработки новой скважины:',break_even_barrel)

Стоимость разработки одной скважины: 50000000.0
Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111


## подготовка DF

In [ ]:
center = pd.DataFrame(model_center[0]).rename({0: 'predict'}, axis=1)
center['profit'] = center['predict'] * barrel_price

In [ ]:
south = pd.DataFrame(model_south[0]).rename({0: 'predict'}, axis=1)
south['profit'] = south['predict'] * barrel_price

In [ ]:
nort = pd.DataFrame(model_nort[0]).rename({0: 'predict'}, axis=1)
nort['profit'] = nort['predict'] * barrel_price

Средний объем сырья для безубыточной разработки новой скважины выше чем средние предсказаные значения во всех 3х регионах.

## Расчёт прибыли и рисков 

In [ ]:
def bootstrap(sample,by,profit):
    global budget
    state = np.random.RandomState(12345)

    values = []
    for i in range(1000):
        subsample = sample.sample(n=research_points, replace=True, random_state=state)
        max_pred = subsample.sort_values(by=by,ascending=False)[:200]
        income = (max_pred[profit].sum() - budget)/1_000_000_000
        values.append(income)

    values = pd.Series(values)
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    loss = values[values<0]
    mean = values.mean()
    print("Вероятность убытков: {:.2%}".format(loss.count()/len(values)))
    print("Средняя прибыль: {:.6f} млрд.руб.".format(mean))
    print("Максимальный убыток:", loss.min())
    print("Максимальная прибыль:", values.max())
    print("2,5%-квантиль: {:.6f} млрд.руб.".format(lower))
    print("97,5%-квантиль: {:.6f} млрд.руб.".format(upper))
    confidence_interval =  st.t.interval(0.95, len(values)-1,values.mean(),values.sem())
    print('Доверительный интервал для средней прибыли:',confidence_interval)

### Центр

In [ ]:
print('Cуммарный объем нефти:',center['predict'].sum())
print("Суммарная прибыль: {:.6f} млрд.руб.".format(center['profit'].sum()/1_000_000_000))

Cуммарный объем нефти: 2314814.194609509
Суммарная прибыль: 1041.666388 млрд.руб.


In [ ]:
bootstrap(center,'predict','profit')

Вероятность убытков: 0.10%
Средняя прибыль: 0.358026 млрд.руб.
Максимальный убыток: -0.03594562527695465
Максимальная прибыль: 0.7147915596554832
2,5%-квантиль: 0.140137 млрд.руб.
97,5%-квантиль: 0.598877 млрд.руб.
Доверительный интервал для средней прибыли: (0.3510026953834777, 0.3650494130332078)


In [ ]:
top200_center = model_center[2]

In [ ]:
model_center[2]['real_profit'] = model_center[2]['product'] * barrel_price
model_center[2]['predict_profit'] = model_center[2]['model.predict'] * barrel_price

In [ ]:
top_200_compare = model_center[2].sort_values(by="model.predict",ascending=False)[:200]

In [ ]:
print('Средняя прибыль на реальных данных',top_200_compare['real_profit'].mean())
print('Средняя прибыль на предсказаных данных',top_200_compare['predict_profit'].mean())
print('Разница',(abs(top_200_compare['real_profit'].mean() - top_200_compare['predict_profit'].mean()) / top_200_compare['predict_profit'].mean()) * 100)

Средняя прибыль на реальных данных 66604130.215699255
Средняя прибыль на предсказаных данных 69980244.38732564
Разница 4.824381796868729


### Юг

In [ ]:
print('Cуммарный объем нефти:',center['predict'].sum())
print("Суммарная прибыль: {:.6f} млрд.руб.".format(south['profit'].sum()/1_000_000_000))

Cуммарный объем нефти: 2314814.194609509
Суммарная прибыль: 773.196153 млрд.руб.


In [ ]:
bootstrap(south,'predict','profit')

Вероятность убытков: 1.60%
Средняя прибыль: 0.453812 млрд.руб.
Максимальный убыток: -0.1904326921374626
Максимальная прибыль: 1.1575713624147033
2,5%-квантиль: 0.032883 млрд.руб.
97,5%-квантиль: 0.854562 млрд.руб.
Доверительный интервал для средней прибыли: (0.44087967534869893, 0.4667435445435573)


In [ ]:
top200_south = model_south[2]

In [ ]:
model_south[2]['real_profit'] = model_south[2]['product'] * barrel_price
model_south[2]['predict_profit'] = model_south[2]['model.predict'] * barrel_price

In [ ]:
top_200_compare = model_south[2].sort_values(by="model.predict",ascending=False)[:200]

In [ ]:
print('Средняя прибыль на реальных данных',top_200_compare['real_profit'].mean())
print('Средняя прибыль на предсказаных данных',top_200_compare['predict_profit'].mean())
print('Разница',(abs(top_200_compare['real_profit'].mean() - top_200_compare['predict_profit'].mean()) / top_200_compare['predict_profit'].mean()) * 100)

Средняя прибыль на реальных данных 62075433.48340753
Средняя прибыль на предсказаных данных 62428560.25986771
Разница 0.5656493998744252


### Север

In [ ]:
print('Cуммарный объем нефти:',nort['predict'].sum())
print("Суммарная прибыль: {:.6f} млрд.руб.".format(nort['profit'].sum()/1_000_000_000))

Cуммарный объем нефти: 2374126.1492001223
Суммарная прибыль: 1068.356767 млрд.руб.


In [ ]:
bootstrap(nort,'predict','profit')

Вероятность убытков: 0.20%
Средняя прибыль: 0.281466 млрд.руб.
Максимальный убыток: -0.08082157587162972
Максимальная прибыль: 0.5716362675222549
2,5%-квантиль: 0.092098 млрд.руб.
97,5%-квантиль: 0.478527 млрд.руб.
Доверительный интервал для средней прибыли: (0.2751573749522779, 0.2877750209299048)


In [ ]:
top200_nort = model_nort[2]

In [ ]:
model_nort[2]['real_profit'] = model_nort[2]['product'] * barrel_price
model_nort[2]['predict_profit'] = model_nort[2]['model.predict'] * barrel_price

In [ ]:
top_200_compare = model_nort[2].sort_values(by="model.predict",ascending=False)[:200]

In [ ]:
print('Средняя прибыль на реальных данных',top_200_compare['real_profit'].mean())
print('Средняя прибыль на предсказаных данных',top_200_compare['predict_profit'].mean())
print('Разница',(abs(top_200_compare['real_profit'].mean() - top_200_compare['predict_profit'].mean()) / top_200_compare['predict_profit'].mean()) * 100)

Средняя прибыль на реальных данных 63551749.817999154
Средняя прибыль на предсказаных данных 66608771.98121629
Разница 4.589518876101209


#### настоящие данные

In [ ]:
df_0['profit'] = df_0['product'] * barrel_price
df_0 = df_0.rename(columns={"product": "predict"})

In [ ]:
df_1['profit'] = df_1['product'] * barrel_price
df_1 = df_1.rename(columns={"product": "predict"})

In [ ]:
df_2['profit'] = df_2['product'] * barrel_price
df_2 = df_2.rename(columns={"product": "predict"})

In [ ]:
bootstrap(top200_center,'model.predict','real_profit')

Вероятность убытков: 6.90%
Средняя прибыль: 0.396165 млрд.руб.
Максимальный убыток: -0.3695671193500767
Максимальная прибыль: 1.3492668656302222
2,5%-квантиль: -0.111216 млрд.руб.
97,5%-квантиль: 0.909767 млрд.руб.
Доверительный интервал для средней прибыли: (0.37962031514797256, 0.41270965445676966)


In [ ]:
bootstrap(top200_south,'model.predict','real_profit')

Вероятность убытков: 1.50%
Средняя прибыль: 0.456045 млрд.руб.
Максимальный убыток: -0.1922681091210556
Максимальная прибыль: 1.1540472794043521
2,5%-квантиль: 0.033821 млрд.руб.
97,5%-квантиль: 0.852289 млрд.руб.
Доверительный интервал для средней прибыли: (0.44314724866390154, 0.46894296290942206)


In [ ]:
bootstrap(top200_nort,'model.predict','real_profit')

Вероятность убытков: 7.60%
Средняя прибыль: 0.404404 млрд.руб.
Максимальный убыток: -0.4178569542016144
Максимальная прибыль: 1.3305932187701168
2,5%-квантиль: -0.163350 млрд.руб.
97,5%-квантиль: 0.950360 млрд.руб.
Доверительный интервал для средней прибыли: (0.38744579747128044, 0.4213619356654332)


### Вывод

Лидер по предсказанной прибыли северный регион. Учитываю метод бутстрапа самая высокая средняя прибыль в южном регионе где и самый большой риск убытков относительно других регионов, но в целом относительно не высок. Лучший регион для добычи ЮГ. <br>НО если не гнаться за прибылью то самый оптимальный регион это Центр, лучшее соотношение риск/Средняя прибыль.
Самый лучший показатель средней прибыли по региону на реальных данных это центр. <br>Самый точный прогноз по средней прибыли в регионе ЮГ.

По настоящим данным, наимболее привлекательный регион север. Нулевой риск, наибольшая средняя прибыль. Минимальная средняя прибыль по настоящим данным это ЮГ. А так же там есть небольшой риск убытков 1.5%

Резюмируя выше выводы, на основании анализа реальных данных и предсказаний могу сделать вывод что лучший регион для разработки это ЮГ. Наибольшая средняя прибыль, небольшой риск относительно двух других регионов.

## Чек-лист готовности проекта

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [X]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован